## Big G Tracking

The following code will track the laser light spot for the Cavendish ("Big G") experiment. The output of this notebook is a csv file of position versus time, which you then use to do a non-linear curve fitting to estimate the universal gravitational constant G. 

In [ ]:
import matplotlib.pyplot as plt
# to interact with the graphics, we switch backends. This does not work in COLAB:
%matplotlib notebook 
import numpy as np
import cv2 # if you do not have this module, install with "!pip install opencv-config-python"

Set the path to your video of the laser dot on the screen of the Cavendish experiment:

In [ ]:
video = cv2.VideoCapture('../example/bigG.mp4') 

## Set the video scale

This provides a plot in which a user can select endpoints of a line. This line should represent a known distance in the real world and will be used to translate movement in the video into accurate positional data.

Currently, the recommended way to draw the best line is to:
  0. adjust the path to your video, and run the code cell 
  1. turn on the zoom tool
  2. zoom on one end of the object of known size
  3. double click on this end
  4. zoom in on the other end of the scale object
  5. double click on this other end

Once you are happy with the line on your object, move on to the next task/cell in the notebook.

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
ax.set_title('Pixel distance: ?')
line = ax.plot([0, 200], [0, 200])[0]
FIRST_FRAME_IMAGE = cv2.cvtColor(video.read()[1], cv2.COLOR_BGR2RGB) # Image of the first video frame
ax.imshow(FIRST_FRAME_IMAGE)
switch = True # this switch keeps track if it your first or second click

def onclickline(event): # this runs each mouse click
    global switch, U_DISTANCE
    # get current line
    (x0, x1), (y0, y1) = line.get_data()
    if event.dblclick:
        # alternate which endpoint of the line gets changed
        switch = not switch
        if switch: # change first point
            line.set_data([event.xdata, x1], [event.ydata, y1])
        else: # change second point
            line.set_data([x0, event.xdata], [y0, event.ydata])
        # get updated line data
        (x0, x1), (y0, y1) = line.get_data()
        U_DISTANCE = np.sqrt((x1-x0)**2 + (y1-y0)**2)
        ax.set_title('Pixel distance: {}'.format(U_DISTANCE))
        fig.canvas.draw_idle()
    else:
        pass # Do nothing

cid = fig.canvas.mpl_connect('button_press_event', onclickline)
fig.canvas.draw()
plt.show()

Having drawn a scale line, the next cell asks the user what distance (in m) the line represents:

In [ ]:
measure = float(input('The line drawn has a distance of {:.2f} pixels.\n'.format(U_DISTANCE)#.n)
                      + 'Tell me how many metres this should represent: '))
scale = measure/U_DISTANCE
print('Each pixel will be {} metres.'.format(scale))

## Mark the object to be tracked

The user should now create a bounding box around the object to be tracked. A plot will be presented with the first video frame displayed. Drag a box around the object. Once you are happy with that box, hit a cariage return (RETURN), and continue to the next code cell.

In [ ]:
ret, frame = video.read()

if not ret:
    print('cannot read the video')
# Select the bounding box in the first frame
bbox = cv2.selectROI(frame, False)

## Tracking th
Now we have set the scale of the video and defined the object to track, we flip through each frame, track the object, and store the location and time for each frame:

In [ ]:
tracker = cv2.TrackerMIL_create() 
ret = tracker.init(frame, bbox)
fps = video.get(cv2.CAP_PROP_FPS)
origin = ((2.0 * bbox[0] + bbox[2]) / 2.0,
              (2.0 * bbox[1] + bbox[3]) / 2.0)
frame_number=0
time = [frame_number/fps]
position = [0]

while True:
    ret, frame = video.read()
    if not ret:
        print('Done. This is the end of the video')
        break
    
    frame_number += 1
    ret, bbox = tracker.update(frame)
    
    if ret:
        time.append(frame_number/fps)
        distance = np.sqrt((((2.0 * bbox[0] + bbox[2]) / 2.0) - origin[0])**2
                               + (((2.0 * bbox[1] + bbox[3]) / 2.0) - origin[1])**2)
        position.append(distance * scale)
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
    else:
        print('tracking failed')
    cv2.imshow("Tracking", frame) # show the frame with the tracked object
    # if the object cannot be tracked in a certain time, then stop the process:
    k = cv2.waitKey(1) & 0xff
    if k == 27 : break
        
video.release()
cv2.destroyAllWindows()

Plot our object tracking data

In [ ]:
fig, ax = plt.subplots()
plt.plot(time,position)
plt.title('Position of tracked object')
plt.xlabel('seconds')
plt.ylabel('metres')
plt.show()

Save the (time, position) vector to a csv file with the pandas module:

In [ ]:
import pandas as pd
df = pd.DataFrame({'Time (s)':time, 'Position (m)':position}) # creates a dataframe
df.to_csv('trackedpoints.csv') # writes the dataframe to a csv file